#### 4.4 Create the state facts table
- Load county facts table
- Discard weather data
- Group by state and timestamp
- Add up case/death total/delta
- Write out to parquet

##### Setup
I'm going to need Spark for this because I'll want to make use of some of its functionality, such as the ability to create temporary SQL views of my dataframes.

In [1]:
from setup import create_spark_session

spark = create_spark_session()

Imports and output paths:

In [2]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import *

from clean import *
from etl import *

# For now, just locally, later on maybe write this to S3 instead
output_path = "output/"

In [3]:
county_facts_df = load_county_facts_table(spark)

In [4]:
county_facts_df.limit(5).show()

+----------+--------------+----------------+----------------+-----------------+-----------------+--------+--------+-----------+----+-----+
| timestamp|         state|covid_case_total|covid_case_delta|covid_death_total|covid_death_delta|min_temp|max_temp|cloud_cover|wind| fips|
+----------+--------------+----------------+----------------+-----------------+-----------------+--------+--------+-----------+----+-----+
|1582156800|North Carolina|               0|               0|                0|                0|     1.0|    11.0|       85.0| 2.0|37181|
|1586390400|North Carolina|              19|               2|                0|                0|    12.0|    28.0|       17.0| 3.0|37181|
|1587945600|North Carolina|              80|               1|                3|                0|     9.0|    22.0|       71.0| 2.0|37181|
|1594080000|North Carolina|             542|               7|               39|                1|    20.0|    33.0|       27.0| 1.0|37181|
|1594512000|North Carolina|

In [5]:
county_facts_df_reduced = county_facts_df[['state', 'timestamp', 'covid_case_total', 'covid_case_delta', 'covid_death_total', 'covid_death_delta']]
county_facts_df_reduced.limit(5).show()

+--------------+----------+----------------+----------------+-----------------+-----------------+
|         state| timestamp|covid_case_total|covid_case_delta|covid_death_total|covid_death_delta|
+--------------+----------+----------------+----------------+-----------------+-----------------+
|North Carolina|1582156800|               0|               0|                0|                0|
|North Carolina|1586390400|              19|               2|                0|                0|
|North Carolina|1587945600|              80|               1|                3|                0|
|North Carolina|1594080000|             542|               7|               39|                1|
|North Carolina|1594512000|             587|              19|               40|                0|
+--------------+----------+----------------+----------------+-----------------+-----------------+



In [6]:
state_facts_df = county_facts_df_reduced.groupBy('state', 'timestamp').agg( \
    F.sum('covid_case_total').alias('covid_case_total'), \
    F.sum('covid_case_delta').alias('covid_case_delta'), \
    F.sum('covid_death_total').alias('covid_death_total'), \
    F.sum('covid_death_delta').alias('covid_death_delta'))

In [7]:
state_facts_df.limit(5).show()

+--------------+----------+----------------+----------------+-----------------+-----------------+
|         state| timestamp|covid_case_total|covid_case_delta|covid_death_total|covid_death_delta|
+--------------+----------+----------------+----------------+-----------------+-----------------+
|North Carolina|1584057600|               1|               1|                0|                0|
|      Arkansas|1587513600|              63|               0|                1|                0|
|      Nebraska|1590019200|              93|               2|                3|                0|
| West Virginia|1586649600|              24|               1|                0|                0|
| West Virginia|1595289600|             477|              13|                5|                0|
+--------------+----------+----------------+----------------+-----------------+-----------------+



In [ ]:
#state_facts_df.write.partitionBy('state').mode('overwrite').parquet(output_path + "state_facts.parquet")